# ADS 509 Module 3: Group Comparison 

The task of comparing two groups of text is fundamental to textual analysis. There are innumerable applications: survey respondents from different segments of customers, speeches by different political parties, words used in Tweets by different constituencies, etc. In this assignment you will build code to effect comparisons between groups of text data, using the ideas learned in reading and lecture.

This assignment asks you to analyze the lyrics and Twitter descriptions for the two artists you selected in Module 1. If the results from that pull were not to your liking, you are welcome to use the zipped data from the “Assignment Materials” section. Specifically, you are asked to do the following: 

* Read in the data, normalize the text, and tokenize it. When you tokenize your Twitter descriptions, keep hashtags and emojis in your token set. 
* Calculate descriptive statistics on the two sets of lyrics and compare the results. 
* For each of the four corpora, find the words that are unique to that corpus. 
* Build word clouds for all four corpora. 

Each one of the analyses has a section dedicated to it below. Before beginning the analysis there is a section for you to read in the data and do your cleaning (tokenization and normalization). 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [3]:
import os
import re
import emoji
import pandas as pd

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from wordcloud import WordCloud 

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer


In [ ]:
# Use this space for any additional import statements you need



In [4]:
# Place any addtional functions or constants you need here. 

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# It's handy to have a full set of emojis
all_language_emojis = set()

for country in emoji.EMOJI_DATA : 
    for em in emoji.EMOJI_DATA[country] : 
        all_language_emojis.add(em)

# and now our functions
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """

    # Place your Module 2 solution here
    
    return(0)


    
def contains_emoji(s):
    
    s = str(s)
    emojis = [ch for ch in s if emoji.is_emoji(ch)]

    return(len(emojis) > 0)


def remove_stop(tokens) :
    # modify this function to remove stopwords
    return(tokens)
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    return(text)

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)


## Data Ingestion

Use this section to ingest your data into the data structures you plan to use. Typically this will be a dictionary or a pandas DataFrame.

In [ ]:
# Feel fre to use the below cells as an example or read in the data in a way you prefer

# Define file path
data_location = "/Users/Sadafvaziri/downloads"
twitter_folder = "twitter/"
full_twitter_path = os.path.join(data_location, twitter_folder)

# Your artist-to-filename mapping
artist_files = {
    'cher': 'cher_followers_data.txt',
    'robyn': 'robynkonichiwa_followers_data.txt'
}

# Read each file into a dictionary
artist_data = {}

for artist, filename in artist_files.items():
    filepath = os.path.join(full_twitter_path, filename)
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            artist_data[artist] = file.readlines()
        print(f"Loaded {artist} data: {len(artist_data[artist])} lines.")
    except FileNotFoundError:
        print(f"File not found: {filepath}")



Loaded cher data: 3994804 lines.
Loaded robyn data: 358373 lines.


In [ ]:
# read in the lyrics here


data_location = "/Users/Sadafvaziri/downloads"
lyrics_folder = "lyrics/"
full_lyrics_path = os.path.join(data_location, lyrics_folder)

# Folder names for each artist
artist_folders = {
    'cher': 'cher',
    'robyn': 'robyn'
}

# Read and combine all lyrics files per artist
lyrics_data = {}

for artist, folder_name in artist_folders.items():
    folder_path = os.path.join(full_lyrics_path, folder_name)
    all_lyrics = []

    try:
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.txt'):
                file_path = os.path.join(folder_path, file_name)
                with open(file_path, 'r', encoding='utf-8') as f:
                    all_lyrics.append(f.read())
        lyrics_data[artist] = "\n".join(all_lyrics)
        print(f"Loaded {len(all_lyrics)} files for {artist}, total {len(lyrics_data[artist])} characters.")
    except FileNotFoundError:
        print(f"Folder not found: {folder_path}")




Loaded 316 files for cher, total 348885 characters.
Loaded 104 files for robyn, total 147880 characters.


## Tokenization and Normalization

In this next section, tokenize and normalize your data. We recommend the following cleaning. 

**Lyrics** 

* Remove song titles
* Casefold to lowercase
* Remove stopwords (optional)
* Remove punctuation
* Split on whitespace

Removal of stopwords is up to you. Your descriptive statistic comparison will be different if you include stopwords, though TF-IDF should still find interesting features for you. Note that we remove stopwords before removing punctuation because the stopword set includes punctuation.

**Twitter Descriptions** 

* Casefold to lowercase
* Remove stopwords
* Remove punctuation other than emojis or hashtags
* Split on whitespace

Removing stopwords seems sensible for the Twitter description data. Remember to leave in emojis and hashtags, since you analyze those. 

In [11]:
# apply the `pipeline` techniques from BTAP Ch 1 or 5

import pandas as pd
import re
from nltk.corpus import stopwords
from string import punctuation

# Define preprocessing components
stop_words = set(stopwords.words("english"))
tw_punct = set(punctuation) - {"#"}
whitespace_pattern = re.compile(r"\s+")

# --- Pipeline functions ---

def remove_punctuation(text, punct_set=tw_punct):
    return "".join([ch for ch in text if ch not in punct_set])

def tokenize(text):
    return whitespace_pattern.split(str(text).strip())

def remove_stop(tokens):
    return [token for token in tokens if token.lower() not in stop_words]

# Prepare function that runs text through the pipeline
def prepare(text, pipeline):
    tokens = str(text)
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens

# --- Define pipeline (BTAP Chapter 1 or 5 style) ---
my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

# --- Load your lyrics_data and artist_data dicts beforehand ---

# Turn lyrics dict into DataFrame
lyrics_df = pd.DataFrame([
    {'artist': artist, 'lyrics': text}
    for artist, text in lyrics_data.items()
])

# Turn twitter dict into DataFrame
twitter_df = pd.DataFrame([
    {'artist': artist, 'description': desc}
    for artist, desc_list in artist_data.items()
    for desc in desc_list
])

# --- Apply pipeline ---

# Lyrics
lyrics_df["tokens"] = lyrics_df["lyrics"].apply(prepare, pipeline=my_pipeline)
lyrics_df["num_tokens"] = lyrics_df["tokens"].map(len)

# Twitter
twitter_df["tokens"] = twitter_df["description"].apply(prepare, pipeline=my_pipeline)
twitter_df["num_tokens"] = twitter_df["tokens"].map(len)

# Optional: preview
print(lyrics_df.head())
print(twitter_df.head())


  artist                                             lyrics  \
0   cher  "Come And Stay With Me"\n\n\n\nI'll send away ...   
1  robyn  "Include Me Out"\n\n\n\nIt is really very simp...   

                                              tokens  num_tokens  
0  [come, stay, ill, send, away, false, pride, il...       35916  
1  [include, really, simple, single, pulse, repea...       15227  
  artist                                        description  \
0   cher  screen_name\tname\tid\tlocation\tfollowers_cou...   
1   cher   hsmcnp\tCountry Girl\t35152213\t\t1302\t1014\t\n   
2   cher  horrormomy\tJeny\t742153090850164742\tEarth\t8...   
3   cher  anju79990584\tanju\t1496463006451974150\t\t13\...   
4   cher     gallionjenna\tJ\t3366479914\t\t752\t556\tcsu\n   

                                              tokens  num_tokens  
0  [screenname, name, id, location, followerscoun...           7  
1      [hsmcnp, country, girl, 35152213, 1302, 1014]           6  
2  [horrormomy, jeny, 7421530

In [13]:
import emoji

def contains_emoji(s):
    s = str(s)
    return any(char in emoji.EMOJI_DATA for char in s)

# Clean up and extract actual description (adjust index if needed)
twitter_df['clean_description'] = twitter_df['description'].apply(lambda x: x.split('\t')[-1].strip())

# Apply emoji detection
twitter_df['has_emoji'] = twitter_df['clean_description'].apply(contains_emoji)

# Preview
print(twitter_df[['artist', 'clean_description', 'has_emoji']].head())



  artist                          clean_description  has_emoji
0   cher                                description      False
1   cher                                                 False
2   cher   𝙿𝚛𝚘𝚞𝚍 𝚜𝚞𝚙𝚙𝚘𝚛𝚝𝚎𝚛 𝚘𝚏 𝚖𝚎𝚜𝚜𝚢 𝚋𝚞𝚗𝚜 & 𝚕𝚎𝚐𝚐𝚒𝚗𝚐𝚜      False
3   cher  163㎝／愛かっぷ💜26歳🍒 工〇好きな女の子💓 フォローしてくれたらDMします🧡       True
4   cher                                        csu      False


Let's take a quick look at some descriptions with emojis.

In [16]:
twitter_df.groupby("artist")["has_emoji"].mean()

artist
cher     0.094505
robyn    0.075246
Name: has_emoji, dtype: float64

In [ ]:


twitter_df[twitter_df['has_emoji'] == True]['clean_description'].value_counts().head(10)


clean_description
❤️      531
🏳️‍🌈    465
❤       351
✨       339
🖤       264
🦋       248
😎       228
✌️      224
♥       218
🌈       206
Name: count, dtype: int64

With the data processed, we can now start work on the assignment questions. 

Q: What is one area of improvement to your tokenization that you could theoretically carry out? (No need to actually do it; let's not make perfect the enemy of good enough.)

A: One area of improvement would be handling and preserving meaningful multi-word expressions (like “New York” or “social media”) as single tokens rather than splitting them into separate words. 

## Calculate descriptive statistics on the two sets of lyrics and compare the results. 


In [17]:
# your code here

from collections import Counter

def descriptive_stats(tokens):
    total = len(tokens)
    unique = len(set(tokens))
    lexical_diversity = unique / total if total > 0 else 0
    most_common = Counter(tokens).most_common(10)
    
    return {
        "Total Tokens": total,
        "Unique Tokens": unique,
        "Lexical Diversity": round(lexical_diversity, 3),
        "Top 10 Tokens": most_common
    }

In [18]:
# Assuming `lyrics_df` has a 'tokens' column and 'artist' column
for artist in lyrics_df['artist'].unique():
    tokens = lyrics_df[lyrics_df['artist'] == artist]['tokens'].values[0]
    stats = descriptive_stats(tokens)
    print(f"\nDescriptive Stats for {artist.title()}:")
    for k, v in stats.items():
        print(f"{k}: {v}")



Descriptive Stats for Cher:
Total Tokens: 35916
Unique Tokens: 3703
Lexical Diversity: 0.103
Top 10 Tokens: [('love', 1004), ('im', 513), ('know', 486), ('dont', 440), ('youre', 333), ('time', 319), ('baby', 319), ('see', 308), ('oh', 306), ('one', 282)]

Descriptive Stats for Robyn:
Total Tokens: 15227
Unique Tokens: 2156
Lexical Diversity: 0.142
Top 10 Tokens: [('know', 308), ('dont', 301), ('im', 299), ('love', 275), ('got', 251), ('like', 232), ('baby', 222), ('youre', 169), ('never', 155), ('dance', 150)]


Q: what observations do you make about these data? 

A: Robyn's lyrics are shorter but more lexically diverse, suggesting a wider vocabulary relative to song length.
 Cher's lyrics repeat emotional themes like “love” and “baby” more frequently, reflecting a more classic pop style.


## Find tokens uniquely related to a corpus

Typically we would use TF-IDF to find unique tokens in documents. Unfortunately, we either have too few documents (if we view each data source as a single document) or too many (if we view each description as a separate document). In the latter case, our problem will be that descriptions tend to be short, so our matrix would be too sparse to support analysis. 

To avoid these problems, we will create a custom statistic to identify words that are uniquely related to each corpus. The idea is to find words that occur often in one corpus and infrequently in the other(s). Since corpora can be of different lengths, we will focus on the _concentration_ of tokens within a corpus. "Concentration" is simply the count of the token divided by the total corpus length. For instance, if a corpus had length 100,000 and a word appeared 1,000 times, then the concentration would be $\frac{1000}{100000} = 0.01$. If the same token had a concentration of $0.005$ in another corpus, then the concentration ratio would be $\frac{0.01}{0.005} = 2$. Very rare words can easily create infinite ratios, so you will also add a cutoff to your code so that a token must appear at least $n$ times for you to return it. 

An example of these calculations can be found in [this spreadsheet](https://docs.google.com/spreadsheets/d/1P87fkyslJhqXFnfYezNYrDrXp_GS8gwSATsZymv-9ms). Please don't hesitate to ask questions if this is confusing. 

In this section find 10 tokens for each of your four corpora that meet the following criteria: 

1. The token appears at least `n` times in all corpora
1. The tokens are in the top 10 for the highest ratio of appearances in a given corpora vs appearances in other corpora.

You will choose a cutoff for yourself based on the side of the corpus you're working with. If you're working with the Robyn-Cher corpora provided, `n=5` seems to perform reasonably well.

In [19]:
# your code here

def get_concentration(tokens):
    total = len(tokens)
    counts = Counter(tokens)
    return {token: count / total for token, count in counts.items()}, counts

def get_top_concentration_ratios(tokens_a, tokens_b, label_a, label_b, n=5, top_k=10):
    conc_a, count_a = get_concentration(tokens_a)
    conc_b, count_b = get_concentration(tokens_b)

    ratios = {}
    for token in conc_a:
        if token in conc_b and count_a[token] >= n and count_b[token] >= n:
            ratio = conc_a[token] / conc_b[token]
            ratios[token] = ratio

    sorted_tokens = sorted(ratios.items(), key=lambda x: x[1], reverse=True)[:top_k]
    print(f"\nTop tokens in {label_a} vs {label_b} (n ≥ {n}):")
    for token, ratio in sorted_tokens:
        print(f"{token}: {round(ratio, 2)}")
    return sorted_tokens


In [20]:
cher_tokens = lyrics_df[lyrics_df['artist'] == 'cher']['tokens'].values[0]
robyn_tokens = lyrics_df[lyrics_df['artist'] == 'robyn']['tokens'].values[0]

# Cher distinctive tokens
get_top_concentration_ratios(cher_tokens, robyn_tokens, 'Cher Lyrics', 'Robyn Lyrics', n=5)

# Robyn distinctive tokens
get_top_concentration_ratios(robyn_tokens, cher_tokens, 'Robyn Lyrics', 'Cher Lyrics', n=5)



Top tokens in Cher Lyrics vs Robyn Lyrics (n ≥ 5):
walk: 10.01
find: 9.68
enough: 7.81
man: 7.53
without: 7.46
strong: 6.53
live: 5.77
believe: 5.59
tears: 4.83
name: 4.41

Top tokens in Robyn Lyrics vs Cher Lyrics (n ≥ 5):
beat: 20.26
dance: 14.15
work: 11.95
hang: 11.2
forgive: 7.55
shake: 7.08
alright: 7.08
party: 6.19
space: 5.9
pretend: 5.9


[('beat', 20.257112945657678),
 ('dance', 14.1522295921718),
 ('work', 11.950771655611742),
 ('hang', 11.203848427136009),
 ('forgive', 7.547855782491627),
 ('shake', 7.0761147960859),
 ('alright', 7.076114796085899),
 ('party', 6.191600446575163),
 ('space', 5.896762330071584),
 ('pretend', 5.896762330071583)]

Q: What are some observations about the top tokens? Do you notice any interesting items on the list? 

A: The top tokens highlight clear thematic differences between Cher and Robyn's lyrics. Cher's list features emotionally weighted and introspective words like "walk," "find," "enough," "tears," and "believe", suggesting a focus on personal struggle, self-reflection, and emotional resilience.

In contrast, Robyn's lyrics are dominated by energetic and physical words like "beat," "dance," "work," "shake," and "party", which emphasize movement, empowerment, and nightlife. The presence of words like "forgive" and "pretend" in Robyn’s list also hints at emotional complexity beneath the surface.

One interesting item is "beat", which is over 20x more concentrated in Robyn’s lyrics—strongly reinforcing her association with electronic/dance music. Similarly, "walk" being the top distinctive token for Cher suggests a more metaphorical or reflective journey that's typical of classic pop ballads.

## Build word clouds for all four corpora. 

For building wordclouds, we'll follow exactly the code of the text. The code in this section can be found [here](https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/ch01/First_Insights.ipynb). If you haven't already, you should absolutely clone the repository that accompanies the book. 


In [ ]:
from matplotlib import pyplot as plt

plt.switch_backend('TkAgg')  # or 'QtAgg' depending on your system


def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400, 
                   background_color= "black", colormap="Paired", 
                   max_font_size=150, max_words=max_words)
    
    # convert data frame into dict
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # filter stop words in frequency counter
    if stopwords is not None:
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)
 
    plt.title(title) 

    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
    
def count_words(df, column='tokens', preprocess=None, min_freq=2):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)
plt.show()  


Q: What observations do you have about these (relatively straightforward) wordclouds? 

A: 